# DeepCube Earth System Data Cube functionalities 
This notebook is a demonstration of some functionalities of the Earth System Data Cube technology  developed in the context of Horizon2020 project DeepCube and prepared for the final review meeting.

The tools are developed in [Julia](https://julialang.org/), a high-performance and versatile scientific programming language. This Jupyter notebook must hence be run with a Julia kernel, preferably with Julia 1.10.0.

In [ ]:
using Pkg; Pkg.activate(".")


If running the notebook for the first time, after activating the environment in the cell above, the cell below needs to be run to instantiate the aforementioned environment in order to install all required packages and their dependencies.

In [ ]:
using Pkg; Pkg.instantiate()

The packages can then be loaded.

In [ ]:
using YAXArrays, Zarr
using OnlineStats: Mean, value, fit!, nobs
using YAXArrays.Cubes: cubesize, formatbytes

For this demo, we rely on [mesogeos](https://zenodo.org/records/7741518), a Mediterranean data cube for the modelling & analysis of wildfires developed in DeepCube. It is stored on the cloud and directly accessible as an s3 bucket.

In [ ]:
ds = open_dataset("https://my-uc3-bucket.s3.gra.io.cloud.ovh.net/mesogeos.zarr")

In [ ]:
# show variables names
 print(keys(ds.cubes))

The dataset contains 30 variables, which all have dimensions x (longitude) and y (latitude). Most of them also have a time dimension.

We select one variable, `burned_areas`, to get information on the size of the cube and its chunking, that is, the way it is stored in compressed files.

In [ ]:
# get dimensions
println("Data cube size: $(size(ds.burned_areas))")
# get chunks
println("Number of chunks: $(size(ds.burned_areas.chunks))")

The dataset is chunked in a way most adapted for spatial analyses: each chunk contains data from one variable over its full spatial extent and only one time step.If we want to analyses more than one time step at a time, we have to load into memory as many chunks as time steps we want to analyse simultaneously. Even if our analysis focuses on a small spatial area, we have to load the full spatial extent into memory.

Hence, for temporal analyses, it is more advisable to rechunk the dataset to chunks with a smaller spatial extent but with more time steps. The rechunking requires to first download the full dataset. It can then be done with YAXArrays.jl in two lines of code as shown below.

In [ ]:
# # DO NOT RUN
# dssub = ds[[:burned_areas,:net_ecosystem_exchange,:leaf_area_index]]

# dschunked = setchunks(dssub,target_chunks)

# savedataset(dschunked,path = "/Net/Groups/BGI/work_3/scratch/fgans/DeepCube/UC3Cube_rechunked2.zarr", max_cache=1e9, backend = :zarr,overwrite = false)
# # max_cache determines the amount of memory to be used for rechunking, the larger this is, the faster the rechunking will go
# # backend can be either :zarr or :netcdf
# # setting overwrite=true will delete any existing dataset
# # setting append=true will append the newly chunked variables to an existing data cube


Once the dataset has been rechunked to serve our purpose, we can access it and perform our temporal analysis.

## Temporal analysis

Our demo aims to analyse to which extent a selection of variables are associated with burned areas. Therefore, we compute the differences between a variable mean over time and over a spatial moving window `(7, 7)` inside burned areas and in their direct surroundings. Since most areas are never burned, we do not need to compute the means at every locations, but only where there was a fire. Hence, we first identify "blobs" of contiguous burned areas in these spatial windows and process them one at a time. That is, we load into memory just the chunks intersecting the spatial window and the duration of the fire event.

In [ ]:
#Open the time series cube
ds = open_dataset("/Net/Groups/BGI/work_3/scratch/fgans/DeepCube/UC3Cube_rechunked2.zarr");

In [ ]:
#Select variables of interest
burned_area = ds.burned_areas;
preds = ("lst_night", "lst_day","dem", "lc_forest", "lc_grassland", "roads_distance")
possible_predictors = map(i->ds[Symbol(i)],preds);


In [ ]:
#Total uncompressed data size:
formatbytes(sum(cubesize,(burned_area,possible_predictors...)))

In [ ]:
indims_burnedarea = InDims(MovingWindow("x",3,3), MovingWindow("y",3,3), "Time", window_oob_value = 0.f0)
indims_predictors = map(possible_predictors) do p
    td = ndims(p) == 3 ? ("Time",) : ()
    InDims(MovingWindow("x",3,3), MovingWindow("y",3,3), td..., window_oob_value = 0.f0)
end

outdims = OutDims(
    Dim{:Variable}(collect(preds)), 
    outtype = Float32, 
    backend=:zarr,
    path = "./output.zarr", 
    overwrite=true
)


In [ ]:
n_workers = 20
threads_per_worker = 16
#Get 20 workers with 32 cpus per worker
using ClusterManagers: SlurmManager
using Distributed
map(1:20) do i
    Threads.@spawn begin
        addprocs(
            SlurmManager(1,fill(2.0,10)),
            partition="big",
            mem_per_cpu="16GB",
            time="00:30:00",
            cpus_per_task=16,
            exeflags=`--project=$(@__DIR__) -t 32 --heap-size-hint=8GB`
            )
    end
end

In [ ]:
#Load code everywhere
@everywhere begin
    using YAXArrays, Zarr
    using OnlineStats: Mean, value, fit!, nobs
    include("windowfire.jl")
    Zarr.Blosc.set_num_threads(16)
end

In [ ]:
mapCube(
    fire_boundaries_window!, 
    (burned_area, possible_predictors...);
    indims = (indims_burnedarea, indims_predictors...), 
    outdims = outdims,
    max_cache=2e9,
)

In [ ]:
rmprocs(workers())

In [ ]:
extrema(filter(i->!ismissing(i) && !isnan(i), data))

In [ ]:
using CairoMakie, Makie, GeoMakie
using YAXArrays, Zarr
ds = open_dataset("output.zarr/")
heatmap(data,colormap=:bluesreds)
data  = reverse(ds.lst_day.data[:,:],dims=2)
heatmap(data,colormap=:bluesreds)